In [1]:
import os
import math
from pathlib import Path

import torch
from safetensors.torch import load_file
from transformers import AutoConfig


def count_parameters_in_safetensors(model_dir: str) -> int:
    """
    Count the total number of parameters stored in one or more .safetensors files
    under a Hugging Face model directory.
    """
    model_path = Path(model_dir)
    if not model_path.exists():
        raise FileNotFoundError(f"Directory not found: {model_dir}")

    # Collect all .safetensors shards
    shards = sorted(model_path.glob("*.safetensors"))
    if not shards:
        raise FileNotFoundError(f"No .safetensors files found in {model_dir}")

    total_params = 0
    for shard in shards:
        state = load_file(str(shard))
        for tensor in state.values():
            total_params += tensor.numel()
        # Free tensors map before loading next shard
        del state
        torch.cuda.empty_cache()

    return total_params


def human_readable(n_params: int) -> str:
    # Return in K, M, B parameters
    if n_params < 1_000:
        return f"{n_params}"
    if n_params < 1_000_000:
        return f"{n_params/1_000:.3f}K"
    if n_params < 1_000_000_000:
        return f"{n_params/1_000_000:.3f}M"
    return f"{n_params/1_000_000_000:.3f}B"


def bytes_on_disk(model_dir: str) -> int:
    total_bytes = 0
    for p in Path(model_dir).glob("*.safetensors"):
        total_bytes += p.stat().st_size
    return total_bytes


def main(model_dir: str):
    # Optional: read config to print model type
    cfg = None
    cfg_path = Path(model_dir) / "config.json"
    if cfg_path.exists():
        try:
            cfg = AutoConfig.from_pretrained(model_dir)
        except Exception:
            cfg = None

    n_params = count_parameters_in_safetensors(model_dir)
    size_bytes = bytes_on_disk(model_dir)

    print("Model directory:", model_dir)
    if cfg is not None:
        print("Model type:", getattr(cfg, "model_type", "unknown"))
        print("Hidden size:", getattr(cfg, "hidden_size", "n/a"))
        print("Num layers:", getattr(cfg, "num_hidden_layers", getattr(cfg, "num_layers", "n/a")))
        print("Num heads:", getattr(cfg, "num_attention_heads", "n/a"))
    print("Total parameters:", n_params, f"({human_readable(n_params)})")

    # Note: on-disk size is compressed (safetensors uses dtype storage). This is not equal to RAM usage.
    print("Total .safetensors size on disk:", f"{size_bytes/1024/1024:.2f} MiB")

/home/yesindeed/anaconda3/envs/vlmbenchmark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
path = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/vqa/SLAKE/intern-vl-8b-lora-ML"

main(path)

Model directory: /media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/vqa/SLAKE/intern-vl-8b-lora-ML
Total parameters: 40370176 (40.370M)
Total .safetensors size on disk: 154.06 MiB
